In [1]:
# Importación de paquetes 
import requests # Consultas de API
import pandas as pd # Manejo de based de datos 
import time # Manejo de tiempo 
from datetime import datetime, timedelta # Manejo de fechas 



Inicialmente habiamos creado un bloque de codigo para traer imformacion de  la Api de coingecko y guardarla en un data frame, para luego hacer un proyecto de series de tiempo en el cual se van a impletar modelos de pronosticos, pero debido a que la Api nos suministra imformación de solo un año, tenemos que crar una automozacion para que me traiga la imfromacion de los años que yo requiera, esto e caso de que nos volvamos usuarios privados de la API, porque para usuarios publicos solo podemos obtenr la imformacion de los ultimos 365 dias, vamos alla.

In [2]:
# Obtener datos de la API coingecko

url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?precision=2"

fecha_i = int(time.mktime(datetime(2023,1,1).timetuple()))  # Fechas Unix
fecha_f = int(time.mktime(datetime(2023,12,1).timetuple()))
# fecha_i = int(time.mktime((datetime.today().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=365)).timetuple()))
# fecha_f = int(time.mktime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0).timetuple()))

# imformación necesaria para consultar con la Api
params = {
    # 'id':'bitcoin',
    'vs_currency':'usd',
    'from': fecha_i,
    'to': fecha_f,
    # 'precision': '2'
}

data = requests.get(url,params=params) # hacer Consulata a la API
data.json().keys()

# Arreglo de la imformacion 
fecha = []
precio_usd = []
for i in data.json()['prices']:
    fecha.append(i[0])
    precio_usd.append(i[1])

market_caps = []
for i in data.json()['market_caps']:
    market_caps.append(i[1])

total_volumes = []
for i in data.json()['total_volumes']:
    total_volumes.append(i[1])

# Construcción de la base de datos 

df = pd.DataFrame({"fecha":fecha,"precio_usd":precio_usd,"market_caps":market_caps,"total_volumes":total_volumes})
df

df["fecha"] = [datetime.fromtimestamp(i/1000).strftime("%d-%m-%Y") for i in df["fecha"]] 

df

KeyError: 'prices'

En este bloque realizamos las automatizaciones necesarias para traer la imfromación, asi que como reusltado tenemos la base de datos del precio del bitcoin en los ultimos 365 dias.

In [3]:
# Primero determinaremos cuantos años necesitamos para nuestro analisis

years = 1  # Incialmente trabajemos con 3, esto quedara automatizado 

bases = {}
# fecha inicial y fecha final(to, from) la fecha inical es la fecha de hoy -364 en fromato unix

fecha_i = int(time.mktime((datetime.today().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=364)).timetuple()))
fecha_f = int(time.mktime(datetime.today().replace(hour=0, minute=0, second=0, microsecond=0).timetuple()))

for j in range(years):
    
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?precision=2" # Url de la Api

    # imformación necesaria para consultar con la Api
    params = {
    # 'id':'bitcoin',
    'vs_currency':'usd',
    'from': fecha_i,
    'to': fecha_f,
    # 'precision': '2'
    }
    data = requests.get(url,params=params) # hacer Consulata a la API

    # print(data.json().keys()) # Nos damos cuenta que el resultado es un diccionario  
    # Con una lista   de lsitas 

    # Arreglo de la imformacion 
    fecha = []
    precio_usd = []
    for i in data.json()['prices']:
        fecha.append(i[0])
        precio_usd.append(i[1])

    market_caps = []
    for i in data.json()['market_caps']:
        market_caps.append(i[1])

    total_volumes = []
    for i in data.json()['total_volumes']:
        total_volumes.append(i[1])

    # Construccion de las base de datos 
    df = pd.DataFrame({"fecha":fecha,"precio_usd":precio_usd,"market_caps":market_caps,"total_volumes":total_volumes})
     
    # Ajuste de la columna fecha con una comorension lista pra darle un formato mas trabajable 
    df["fecha"] = [datetime.fromtimestamp(i/1000).strftime("%d-%m-%Y") for i in df["fecha"]] 
    
    # Agregamos la base de datos de cada iteración al diccionario 

    bases[f"Base_{j}"] = df 
    
    # Actualizamos las fechas para la siguiente iteración
    fecha_f = fecha_i
    fecha_i = int(time.mktime((datetime.fromtimestamp(fecha_f) - timedelta(days=365)).timetuple()))
    

In [6]:
df = bases['Base_0']
df.head()

,fecha,precio_usd,market_caps,total_volumes
0,28-08-2024,59015.30,1.166032e+12,4.201899e+10
1,29-08-2024,59351.58,1.171762e+12,3.256333e+10
2,30-08-2024,59156.25,1.168247e+12,4.403850e+10
3,31-08-2024,58960.24,1.164278e+12,1.139909e+10
4,01-09-2024,57357.72,1.133133e+12,2.546562e+10


Ahora continuamos con el procesamiento de la imformación, revisamos que no existan datos nulos o faltantes, que no existan duplicados en las fechas y testear que transformaciones podemos realizar a las variables que tenemos en nuestra base de datos.

In [8]:
df.describe()
df.dtypes


fecha             object
precio_usd       float64
market_caps      float64
total_volumes    float64
dtype: object